### импорты и загрузка теста

In [ ]:
# %%capture
!pip install open-clip-torch
!pip install decord
!pip install -U bitsandbytes
import open_clip

In [27]:
import torch
from PIL import Image
from tqdm import tqdm
import json
import pandas as pd
import open_clip

In [28]:
#test
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/fill_in_blank_test.json') as f:
    test_data = json.load(f)

with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/test.json') as f:
    test_items = json.load(f)
test_df = pd.DataFrame(test_items)

In [29]:
#val
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/fill_in_blank_valid.json') as f:
    val_data = json.load(f)

with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/valid.json') as f:
    val_items = json.load(f)
val_df = pd.DataFrame(val_items)

In [30]:
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/polyvore_item_metadata.json') as f:
    item_data = json.load(f)

In [ ]:
#train
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/fill_in_blank_train.json') as f:
    train_data = json.load(f)

with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/train.json') as f:
    train_items = json.load(f)
train_df = pd.DataFrame(train_items)
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/polyvore_item_metadata.json') as f:
    item_data = json.load(f)
    
indexes = []
from tqdm import tqdm

for i in tqdm(range(len(test_data))):
    outfit_id = train_data[i]['question'][0].split('_')[0]
    items_list = train_df.loc[test_df['set_id']==outfit_id,'items'].tolist().copy()

    for j in items_list[0]:
        indexes.append(j['item_id'])
    

100%|██████████| 15145/15145 [00:23<00:00, 648.05it/s]


In [32]:
len(indexes),len(set(indexes))

(74262, 70035)

для начала проверим качество моделей на Fill-in-the-Blank Prediction датасета polyvore_outfits

### попробуем clip

In [ ]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:Marqo/marqo-fashionCLIP')
tokenizer = open_clip.get_tokenizer('hf-hub:Marqo/marqo-fashionCLIP')

model.cuda()
pass

In [29]:
BATCH_SIZE = 2048 

item_ids = list(set(indexes))
emb_list = []

with torch.no_grad(), torch.cuda.amp.autocast():
    for batch_idx in tqdm(range(0, len(item_ids), BATCH_SIZE)):
        batch = item_ids[batch_idx:batch_idx + BATCH_SIZE]
        
        image_batch = []
        for item_id in batch:
            img_path = f"/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/{item_id}.jpg"
            image = preprocess_val(Image.open(img_path))
            image_batch.append(image)
        
        if image_batch:
            images = torch.stack(image_batch).cuda()
            img_embeddings = model.encode_image(images, normalize=True).cpu()
            
            for i, item_id in enumerate(batch):
                emb_list.append({'item_id':item_id,'img_emb':img_embeddings[i]})

/tmp/ipykernel_329/1756477911.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
100%|██████████| 35/35 [14:42<00:00, 25.20s/it]


In [70]:
emb_dict = {}
for i in emb_list:
    emb_dict[i['item_id']] = i['img_emb']

In [ ]:
img_emb = pd.DataFrame(emb_list)

In [ ]:
img_emb['img_emb'] = np.array(img_emb['img_emb'].tolist()).tolist()

In [40]:
img_emb.to_parquet('test_img_emb.parquet',index=False)

In [89]:
from IPython.display import display
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import torch.nn.functional as F
result = 0

for i in tqdm(range(len(test_data))):
    outfit_id = test_data[i]['question'][0].split('_')[0]
    items_list = [j['item_id'] for j in test_df.loc[test_df['set_id']==outfit_id,'items'].tolist().copy()[0]]
    target = test_data[i]['blank_position']

    target_item = items_list[target-1]
    items_list.pop(target-1)
    
    items_embs = torch.Tensor(np.array([emb_dict[j] for j in items_list]))
    items_embs = F.normalize(items_embs, p=2, dim=1)

    similarities = []
    answers = []
    for j in test_data[i]['answers']:
        
        outfit_id, index = j.split('_')

        elem = test_df.loc[test_df['set_id']==outfit_id,'items'].tolist().copy()
        elem = elem[0][int(index)-1]['item_id']
        answers.append(elem)
        
        sim = torch.mm(emb_dict[elem].unsqueeze(0), items_embs.T)
        similarities.append(sim.max().item())
        
    pred = np.argmax(similarities)
    
    if(answers[pred]==target_item):
        result += 1


100%|██████████| 15145/15145 [02:03<00:00, 122.47it/s]


In [90]:
result/len(test_data)

0.5247276328821393

In [ ]:
marqo-fashionCLIP (only img) - mean KNN: 0.56956 
marqo-fashionCLIP (only img) - max  KNN: 0.52472 

### далее попробуем делать рекомендацию с помощью VLM
за основу возьмем InterVL3 8B

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel,AutoConfig
import math
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
import os

In [ ]:

def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained("OpenGVLab/InternVL3-8B", trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.model.rotary_emb'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

path = "OpenGVLab/InternVL3-8B"
device_map = split_model('InternVL3-8B')
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=False,
    use_flash_attn=False,
    trust_remote_code=True,
    device_map=device_map).eval()

In [ ]:
# дефолтные преобразования internvl
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [ ]:
indexes = []
from tqdm import tqdm

for i in tqdm(range(len(val_data))):
    outfit_id = val_data[i]['question'][0].split('_')[0]
    items_list = val_df.loc[val_df['set_id']==outfit_id,'items'].tolist().copy()

    for j in items_list[0]:
        indexes.append(j['item_id'])


100%|██████████| 3000/3000 [00:01<00:00, 1981.64it/s]


In [54]:
len(indexes),len(set(indexes))

(15288, 14657)

In [ ]:
from lmdeploy import pipeline, TurbomindEngineConfig
from lmdeploy.vl import load_image
from lmdeploy.vl.constants import IMAGE_TOKEN
import json
from tqdm import tqdm

path = 'OpenGVLab/InternVL3-8B'
pipe = pipeline(path, backend_config=TurbomindEngineConfig(session_len=8192, tp=4))

path2img = '/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/'
batch_size = 4  

def process_batch(batch_data):
    prompts = []
    images_batch = []
    
    for data in batch_data:
        outfit_id = data['question'][0].split('_')[0]
        items_list = [j['item_id'] for j in val_df.loc[val_df['set_id']==outfit_id,'items'].tolist().copy()[0]]
        target = data['blank_position']
        target_item = items_list[target-1]
        items_list.pop(target-1)
        
        answers = []
        for j in data['answers']:
            outfit_id, index = j.split('_')
            elem = val_df.loc[val_df['set_id']==outfit_id,'items'].tolist().copy()
            elem = elem[0][int(index)-1]['item_id']
            answers.append(elem)
        
        sem_cat = item_data[str(answers[0])]['semantic_category']
        items_img_paths = [path2img+j+'.jpg' for j in items_list]
        
        try:
            imgs = [load_image(img_path) for img_path in items_img_paths]
            imgs_tokens = IMAGE_TOKEN * len(imgs)
            
            prompt = f'''
            You are a shopping assistant who recommends an item of clothing for an image.
            the image consists of these clothing elements: {imgs_tokens}
            You should suggest another piece of clothing from the semantic category: {sem_cat}.
            Make a brief description of the item of clothing that would perfectly match this image.
            the response should not contain references to images from the image such as "an item of clothing suitable for the image.".
            Your answer should look like this:
            '''+'''{'description':'item description'}'''
            
            prompts.append(prompt)
            images_batch.append(imgs)
        except Exception as e:
            print(f"Error loading images: {e}")
            continue
    
    if prompts and images_batch:
        responses = pipe(list(zip(prompts, images_batch)))
        return responses
    return []

results = []
for i in tqdm(range(0, len(val_data), batch_size)):
    batch = val_data[i:i+batch_size]
    batch_results = process_batch(batch)
    results.extend(batch_results)


In [28]:
for i in range(10):
    print(results[1+i].text)

{'description':'A pair of elegant, silver hoop earrings with intricate detailing, complementing the bold and stylish accessories in the image.'}
{'description':'A black leather moto jacket with a studded design and a cropped fit, featuring a collar and zippered front for a bold and edgy look.'}
{'description': 'A stylish green quilted handbag with a gold emblem and adjustable straps, complementing the vibrant floral patterns and bold colors of the raincoat and skirt in the image.'}
{'description': 'A delicate pink pendant necklace featuring an intricate floral design that complements the floral pattern on the skirt, adding a touch of elegance and harmony to the overall look.'}
{'description': 'A sleek black crossbody bag with white accents to complement the black and white color scheme of the outfit.'}
{'description': 'A pair of elegant olive green ankle boots with a sleek design and a slight heel, complementing the chic and sophisticated look of the dress.'}
{'description': 'A beige c

In [ ]:
answers = [i.text for i in results]

In [19]:
answers_df = pd.DataFrame(answers,columns=['descrtiption'])
answers_df.head()
answers_df.to_csv('answers22.csv',index=False)

In [8]:
answers_df = pd.read_csv('/kaggle/working/answers.csv')

In [9]:
import re
def fix_single_quoted_json(json_str):

    fixed_str = json_str

    if (fixed_str[0]=='"'): 
        fixed_std = fixed_std[1:]
    if (fixed_str[-1]=='"'): 
        fixed_std = fixed_std[:-1]

    fixed_str = fixed_str.replace('"', "'")
    fixed_str = fixed_str.replace("'", '"', 1)  
    fixed_str = fixed_str.replace("'", '"', 1)
    fixed_str = fixed_str.replace("'", '"', 1)
    fixed_str = fixed_str[::-1].replace("'", '"', 1)[::-1]  
    
    return json.loads(fixed_str)['description']


In [10]:
answers_df['description_text'] = answers_df['descrtiption'].apply(lambda x: fix_single_quoted_json(x))
answers_df

,descrtiption,description_text
0,{'description':'Green and black camouflage-pat...,Green and black camouflage-patterned ankle boo...
1,"{'description':'A pair of elegant, silver hoop...","A pair of elegant, silver hoop earrings with i..."
2,{'description':'A black leather moto jacket wi...,A black leather moto jacket with a studded des...
3,{'description': 'A stylish green quilted handb...,A stylish green quilted handbag with a gold em...
4,{'description': 'A delicate pink pendant neckl...,A delicate pink pendant necklace featuring an ...
...,...,...
2995,"{'description':'A chic, metallic gold blouse w...","A chic, metallic gold blouse with a tailored f..."
2996,"{'description': 'High-waisted, wide-leg trouse...","High-waisted, wide-leg trousers in a neutral b..."
2997,{'description': 'A black leather jacket with a...,"A black leather jacket with a sleek, modern de..."
2998,"{'description': 'A pair of high-waisted, black...","A pair of high-waisted, black leather pants wi..."


In [ ]:
path2img = '/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/'
accuracy = 0
device = 'cuda'
with torch.no_grad(), torch.cuda.amp.autocast():
    for i in tqdm(range(len(val_data))):
        outfit_id = val_data[i]['question'][0].split('_')[0]
        items_list = [j['item_id'] for j in val_df.loc[val_df['set_id']==outfit_id,'items'].tolist().copy()[0]]
        target = val_data[i]['blank_position']
    
        target_item = items_list[target-1]
        items_list.pop(target-1)
    
        similarities = []
        answers = []
        for j in val_data[i]['answers']:
            outfit_id, index = j.split('_')
            elem = val_df.loc[val_df['set_id']==outfit_id,'items'].tolist().copy()
            elem = elem[0][int(index)-1]['item_id']
            answers.append(elem)
    
        llm_text = answers_df.loc[i,'description_text']
        answers_texts = [item_data[j]['url_name'] for j in answers]
        answers_img_paths = [path2img+j+'.jpg' for j in answers]
        
        text_inputs = tokenizer([llm_text] + answers_texts).to(device)

        image_features = []
        for img_path in answers_img_paths:
            try:
                image = preprocess_val(Image.open(img_path)).unsqueeze(0).to(device)
                with torch.no_grad(), torch.cuda.amp.autocast():
                    features = model.encode_image(image, normalize=True)
                    image_features.append(features)
            except:
                print('gg')
                image_features.append(torch.zeros(1, model.visual.output_dim).to(device))
        
        if not image_features:
            continue
        
        image_features = torch.cat(image_features, dim=0)
        
        with torch.no_grad(), torch.cuda.amp.autocast():
            text_features = model.encode_text(text_inputs, normalize=True)
        
        desc_features = text_features[0:1]
        ans_text_features = text_features[1:]
        

        image_to_desc_sim = (100.0 * desc_features @ image_features.T).softmax(dim=-1)
        
        text_to_desc_sim = (100.0 * desc_features @ ans_text_features.T).softmax(dim=-1)
        
        image_to_text_sim = (100.0 * ans_text_features @ image_features.T).diag()
        
        combined_sim = text_to_desc_sim  
        predict_item = answers[combined_sim.argmax().item()]
        
        if predict_item == target_item:
            accuracy += 1

print(f"Final accuracy: {accuracy / len(val_data):.4f}")

/tmp/ipykernel_395/1434902131.py:4: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
  0%|          | 0/3000 [00:00<?, ?it/s]/tmp/ipykernel_395/1434902131.py:34: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
/tmp/ipykernel_395/1434902131.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():
100%|██████████| 3000/3000 [03:09<00:00, 15.80it/s]

Final accuracy: 0.3227


In [ ]:
Final accuracy: 0.4673 - only img
Final accuracy: 0.3227 - only text

vlm плохо справляется с рекомендациями, изменим задачу, теперь будем брать айтем, делать к нему текстовую рекомендацию, затем смотреть, попадает ли айтем в топ 1, 5, 25, 100 по симилярити к хоть какому-то из релевантных сэмплу
это более актуальная задача, мы рекомендуем по айтему оставшийся образ, а не по образу заполняем айтем, это ключевой различие подходов FITB и рекомендации образа по айтему

### создание описаний элементов трейна

In [37]:
!git clone https://github.com/OpenGVLab/InternVL

Cloning into 'InternVL'...
remote: Enumerating objects: 3549, done.
remote: Counting objects: 100% (425/425), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 3549 (delta 358), reused 282 (delta 282), pack-reused 3124 (from 4)
Receiving objects: 100% (3549/3549), 39.51 MiB | 38.56 MiB/s, done.
Resolving deltas: 100% (2123/2123), done.


In [1]:
!pip uninstall bitsandbytes -y
!pip install bitsandbytes

Found existing installation: bitsandbytes 0.41.0
Uninstalling bitsandbytes-0.41.0:
  Successfully uninstalled bitsandbytes-0.41.0
  Using cached bitsandbytes-0.46.0-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.46.0-py3-none-manylinux_2_24_x86_64.whl (67.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
internvl-chat 2.0.0 requires bitsandbytes==0.41.0, but you have bitsandbytes 0.46.0 which is incompatible.


In [3]:
from lmdeploy import pipeline, TurbomindEngineConfig, ChatTemplateConfig
from lmdeploy.vl import load_image

model = 'OpenGVLab/InternVL3-8B'
pipe = pipeline(model, backend_config=TurbomindEngineConfig(session_len=32768, tp=4), chat_template_config=ChatTemplateConfig(model_name='internvl2_5'))



Fetching 24 files:   0%|          | 0/24 [00:00<?, ?it/s]

2025-05-25 15:04:32.265239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748185472.287473    1324 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748185472.294207    1324 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


FlashAttention2 is not installed.


  0%|          | 0/431 [00:00<?, ?w/s]

  0%|          | 0/123 [00:00<?, ?w/s]

  0%|          | 0/124 [00:00<?, ?w/s]

  0%|          | 0/7 [00:00<?, ?w/s]

2025-05-25 15:04:40,271 - lmdeploy - WARNING - turbomind.py:252 - get 849 model params


[TM][WARNING] [LlamaTritonModel] `max_context_token_num` is not set, default to 32768.
[TM][WARNING] pad vocab size from 151674 to 151676
[TM][WARNING] pad embed size from 151676 to 151676
[TM][WARNING] pad vocab size from 151674 to 151676
[TM][WARNING] pad embed size from 151676 to 151676
[TM][WARNING] pad vocab size from 151674 to 151676
[TM][WARNING] pad embed size from 151676 to 151676
[TM][WARNING] pad vocab size from 151674 to 151676
[TM][WARNING] pad embed size from 151676 to 151676


2025-05-25 15:04:47,900 - lmdeploy - WARNING - tokenizer.py:499 - The token <|action_end|>, its length of indexes [27, 91, 1311, 6213, 91, 29] is over than 1. Currently, it can not be used as stop words


In [1]:
pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 117.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
# дефолтные преобразования для intervl 
def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


In [ ]:
import math
import torch
from transformers import AutoTokenizer, AutoModel,AutoConfig

def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.model.rotary_emb'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

path = "/kaggle/working/internvl2_5_4b_dynamic_res_2nd_finetune_lora_merged"
device_map = split_model('OpenGVLab/InternVL3-8B')
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map=device_map).eval()


In [12]:
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
pip install lmdeploy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 MB 22.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.9 MB/s et

In [ ]:

import json
import pandas as pd
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/fill_in_blank_train.json') as f:
    train_data = json.load(f)

with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/train.json') as f:
    train_items = json.load(f)
train_df = pd.DataFrame(train_items)
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/polyvore_item_metadata.json') as f:
    item_data = json.load(f)
    
indexes = []
from tqdm import tqdm

for i in tqdm(range(len(train_data))):
    outfit_id = train_data[i]['question'][0].split('_')[0]
    items_list = train_df.loc[train_df['set_id']==outfit_id,'items'].tolist().copy()

    for j in items_list[0]:
        indexes.append(j['item_id'])


100%|██████████| 16995/16995 [00:27<00:00, 617.85it/s]


In [78]:
len(list(set(indexes)))

71967

In [ ]:
from lmdeploy import pipeline, TurbomindEngineConfig
from lmdeploy.vl import load_image
from lmdeploy.vl.constants import IMAGE_TOKEN
import json
from tqdm import tqdm
import os

path = 'OpenGVLab/InternVL3-8B'
pipe = pipeline(path, backend_config=TurbomindEngineConfig(session_len=8192, tp=4))

path2img = '/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/'
batch_size = 4 

In [ ]:

def process_images(index_set):
    results = {}
    
    indices = list(index_set)
    
    for i in tqdm(range(0, len(indices), batch_size)):
        batch_indices = indices[i:i+batch_size]
        prompts = []
        images_batch = []
        current_batch = []
        
        for index in batch_indices:
            img_path = os.path.join(path2img, f"{index}.jpg")
            
            try:
                img = load_image(img_path)  
                prompt = f"{IMAGE_TOKEN}\nDescribe this item of clothing in detail, including its type, style, color, do not add unnecessary words, just a description. start your text with This is a"
                
                prompts.append(prompt)
                images_batch.append([img])  
                current_batch.append(index)
            except Exception as e:
                print(f"Error loading image {index}: {e}")
                continue
        
        if prompts and images_batch:
            responses = pipe(list(zip(prompts, images_batch)))
            
            for idx, response in zip(current_batch, responses):
                results[idx] = response.text  
    
    return results

vlm_responses = process_images(list(set(indexes))[30000:40500])


100%|██████████| 2625/2625 [1:04:05<00:00,  1.47s/it]


In [81]:
with open('data_sorted6.json', 'w', encoding='utf-8') as f:
    json.dump(vlm_responses, f, ensure_ascii=False, indent=4, sort_keys=True)

## sft dataset


In [15]:
import pandas as pd
import json
desc = []
for i in range(1,10):
    path = f'/kaggle/input/description-data/data_sorted{i}.json'
    if (i==1):
        path = f'/kaggle/input/description-data/data_sorted.json'
    with open(path, 'r', encoding='utf-8') as f:
        desc.append( pd.DataFrame(json.load(f),index = ['description']).T)
desc_df = pd.concat(desc)
desc_df

,description
100047801,"This is a white, textured pouch with a zipper ..."
100279451,"Gold-colored bangle bracelet with a textured, ..."
100338720,This is a gold-toned necklace with a pendant f...
100426698,The item is a pair of sunglasses. They have a ...
100503275,The item is a gold-colored high-heeled sandal ...
...,...
99776051,This is a diamond engagement ring featuring a ...
99853799,This is a pair of black winter boots. They hav...
99888749,This is a pair of black velvet loafers. They f...
99923242,This is a pair of brown leather work boots. Th...


In [16]:
desc_df['start_word'] = desc_df.description.apply(lambda x: ' '.join(x.split(' ')[:3]))

In [17]:
desc_df['start_word'].value_counts()[:50]

start_word
This is a                       64786
The item is                      5617
The image shows                   287
The item of                       159
This is an                        143
This item is                       90
Sunglasses, aviator style,         43
Light blue denim                   25
These are black                    24
Black leather jacket               23
Straw hat with                     22
Gold ring with                     20
These are gold                     18
Two-piece swimsuit with            15
Straw hat, wide                    15
The clothing is                    15
Gold-colored bangle bracelet       13
Black skinny jeans                 12
Straw fedora hat                   12
These are women's                  10
Gold-colored bracelet with         10
Gold bracelet with                 10
These are a                         9
These are beige                     8
These are hoop                      7
Black denim shorts                  7
B

In [18]:
to_del = ['The item is','This is a','The image shows','The item of','This item is','This is an','These are a']

In [21]:
desc_df['description1'] = desc_df.description.apply(lambda x: ' '.join(x.split(' ')[3:] if (' '.join(x.split(' ')[:3]) in to_del) else ''))
desc_df.loc[desc_df['description1'] == '','description1'] = desc_df.loc[desc_df['description1'] == '','description']

In [22]:
desc_df.head(20)

,description,start_word,description1
100047801,"This is a white, textured pouch with a zipper ...",This is a,"white, textured pouch with a zipper closure. I..."
100279451,"Gold-colored bangle bracelet with a textured, ...",Gold-colored bangle bracelet,"Gold-colored bangle bracelet with a textured, ..."
100338720,This is a gold-toned necklace with a pendant f...,This is a,gold-toned necklace with a pendant featuring a...
100426698,The item is a pair of sunglasses. They have a ...,The item is,a pair of sunglasses. They have a tortoiseshel...
100503275,The item is a gold-colored high-heeled sandal ...,The item is,a gold-colored high-heeled sandal with an ankl...
100532766,The item is a pair of mustard yellow cargo sho...,The item is,a pair of mustard yellow cargo shorts. They fe...
100871728,This is a women's sandal featuring a brown lea...,This is a,women's sandal featuring a brown leather desig...
100875976,This is a women's sandal with a flat sole and ...,This is a,women's sandal with a flat sole and a wooden h...
101124203,"This is a sleeveless, floor-length dress with ...",This is a,"sleeveless, floor-length dress with thin strap..."
101178943,"This is a sleeveless, heather gray cropped swe...",This is a,"sleeveless, heather gray cropped sweater with ..."


In [91]:
desc_df.to_csv('filtered_desc_df2.csv',index = False)

In [10]:
with open('data_sorted2.json', 'w', encoding='utf-8') as f:
    json.dump(vlm_responses, f, ensure_ascii=False, indent=4, sort_keys=True)

In [2]:
import json
with open('/kaggle/working/data_sorted.json', 'r', encoding='utf-8') as f:
    desc = json.load(f)

In [3]:
import pandas as pd
desc_df = pd.read_csv('filtered_desc_df2.csv')

In [23]:
desc_filt = desc_df.reset_index()

In [24]:
len(set(desc_filt['index']))

48742

In [25]:
desc_filt

,index,description,start_word,description1
0,100047801,"This is a white, textured pouch with a zipper ...",This is a,"white, textured pouch with a zipper closure. I..."
1,100279451,"Gold-colored bangle bracelet with a textured, ...",Gold-colored bangle bracelet,"Gold-colored bangle bracelet with a textured, ..."
2,100338720,This is a gold-toned necklace with a pendant f...,This is a,gold-toned necklace with a pendant featuring a...
3,100426698,The item is a pair of sunglasses. They have a ...,The item is,a pair of sunglasses. They have a tortoiseshel...
4,100503275,The item is a gold-colored high-heeled sandal ...,The item is,a gold-colored high-heeled sandal with an ankl...
...,...,...,...,...
71962,99776051,This is a diamond engagement ring featuring a ...,This is a,"diamond engagement ring featuring a large, rec..."
71963,99853799,This is a pair of black winter boots. They hav...,This is a,pair of black winter boots. They have a high-t...
71964,99888749,This is a pair of black velvet loafers. They f...,This is a,pair of black velvet loafers. They feature a p...
71965,99923242,This is a pair of brown leather work boots. Th...,This is a,pair of brown leather work boots. They have a ...


In [26]:
desc_filt = desc_filt.drop_duplicates(subset=['index'],keep='last')

In [27]:
index_list = desc_filt['index'].tolist()

In [ ]:
import json
import pandas as pd
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/fill_in_blank_train.json') as f:
    train_data = json.load(f)

with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/train.json') as f:
    train_items = json.load(f)
train_df = pd.DataFrame(train_items)
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/polyvore_item_metadata.json') as f:
    item_data = json.load(f)
    
outfits = []
from tqdm import tqdm

for i in tqdm(range(len(train_data))):
    outfit_id = train_data[i]['question'][0].split('_')[0]
    items_list = train_df.loc[train_df['set_id']==outfit_id,'items'].tolist().copy()
    desc_items = []
    for j in items_list[0]:
        if (j['item_id'] in index_list):
            desc_items.append(j)
    outfits.append({'id':outfit_id,'items':desc_items})


100%|██████████| 16995/16995 [01:16<00:00, 222.91it/s]


In [ ]:

good_outfits = []
for i in outfits:
    if (len(i['items']) > 2):
        good_outfits.append(i)


12365

In [104]:
good_outfits[0]

{'id': '199244701',
 'items': [{'item_id': '132621870', 'index': 1},
  {'item_id': '153967122', 'index': 2},
  {'item_id': '171169800', 'index': 3},
  {'item_id': '162799044', 'index': 4},
  {'item_id': '172312529', 'index': 6}]}

In [188]:
def get_prompt(n,desc):
    return f'''you are a shopping assistant. recommend {n} items of clothing for {desc} , you must analyze the description of the clothing, its image.
 in the answer you must make a detailed description of things that match this item of clothing '''

In [176]:
def get_gpt_response(item_list):
    text = 'goes with this item of clothing: ['
    for i in range(len(item_list)):
        text+="'"+desc_filt.loc[desc_filt['index']==item_list[i]['item_id'],'description1'].tolist()[0] +"', "
    return text[:-2]+']'

In [ ]:
import random
# "conversations": [{"from": "human", "value": ""}, {"from": "gpt", "value": ""}]}

sft_set = []
for i in tqdm(range(len(good_outfits))):
    query_id = random.randint(0,len(good_outfits[i]['items'])-1)
    item_list = good_outfits[i]['items'].copy()
        
    query_item = item_list[query_id]
    item_list.pop(query_id)
    human_request = get_prompt(len(item_list),desc_filt.loc[desc_filt['index']==query_item['item_id'],'description1'].tolist()[0])
    gpt_response = get_gpt_response(item_list)
    conversations = [{"from":"human",'value':human_request},{'from':'gpt','value':gpt_response}]
    sft_set.append({'id':good_outfits[i]['id'],'image':str(query_item['item_id'])+'.jpg','conversations':conversations})

100%|██████████| 12365/12365 [03:08<00:00, 65.50it/s]


In [193]:
sft_text = '\n'.join([json.dumps(sft_set[i]) for i in range(len(sft_set))])

In [195]:

with open('/kaggle/working/InternVL/internvl_chat/internvl/outfit_dataset.jsonl','w') as f:
        f.write(sft_text)

In [196]:

with open('/kaggle/working/InternVL/internvl_chat/internvl/outfit_dataset.jsonl','r') as f:
        texts = f.readlines()

In [198]:
json.loads(texts[0])

{'id': '199244701',
 'image': '153967122.jpg',
 'conversations': [{'from': 'human',
   'value': 'you are a shopping assistant. recommend 4 items of clothing for black high-heeled sandal with a pointed toe. It features a thin ankle strap and a decorative buckle on the upper strap. The insole is nude-colored, and the heel is stiletto-style. , you must analyze the description of the clothing, its image.\n in the answer you must make a detailed description of things that match this item of clothing '},
  {'from': 'gpt',
   'value': "goes with this item of clothing: ['structured clutch purse with a rectangular shape. It features a textured, metallic silver surface with a diamond pattern. The top of the purse has a unique clasp mechanism with two rectangular elements. The edges are framed with a thin, metallic border.', 'silver pendant featuring a teardrop-shaped top resembling a flame, attached to a textured spherical body. It is connected to a delicate chain with a small, simple clasp.', '

In [199]:
len(texts)

12365

In [200]:
json_ann = '''
{
  "my_dataset": {
    "root": "/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/",
    "annotation": "internvl/outfit_dataset.jsonl",
    "data_augment": false,
    "repeat_time": 1,
    "length": 12365
  }
}
'''
with open('/kaggle/working/InternVL/internvl_chat/internvl/custom_outfit_ann.json','w') as f:
    f.write(json_ann)

### обучение InternVL3-8B с lora

логи обучения я загрузил в репозиторий

In [ ]:
%%capture
!pip install -r /kaggle/working/InternVL/requirements/internvl_chat.txt
!pip uninstall bitsandbytes -y
!pip install bitsandbytes
!pip install flash-attn
!pip install decord

cd /kaggle/working/InternVL/internvl_chat

In [4]:
%%bash
set -x

GPUS=${GPUS:-4}
BATCH_SIZE=${BATCH_SIZE:-4}
PER_DEVICE_BATCH_SIZE=${PER_DEVICE_BATCH_SIZE:-1}
GRADIENT_ACC=$((BATCH_SIZE / PER_DEVICE_BATCH_SIZE / GPUS))


export PYTHONPATH="${PYTHONPATH}:$(pwd)"
export MASTER_PORT=34229
export TF_CPP_MIN_LOG_LEVEL=3
export LAUNCHER=pytorch

OUTPUT_DIR='/kaggle/agi/internvl_chat_v2_5/internvl2_5_4b_dynamic_res_2nd_finetune_lora'

if [ ! -d "$OUTPUT_DIR" ]; then
  mkdir -p "$OUTPUT_DIR"
fi

# number of gpus: 2
# batch size per gpu: 4
# gradient accumulation steps: 2
# total batch size: 16
# epoch: 1
torchrun \
  --nnodes=1 \
  --node_rank=0 \
  --master_addr=127.0.0.1 \
  --nproc_per_node=${GPUS} \
  --master_port=${MASTER_PORT} \
  /kaggle/working/InternVL/internvl_chat/internvl/train/internvl_chat_finetune.py \
  --model_name_or_path "OpenGVLab/InternVL3-8B" \
  --conv_style "internvl2_5" \
  --use_fast_tokenizer False \
  --output_dir ${OUTPUT_DIR} \
  --meta_path "internvl/custom_outfit_ann.json" \
  --overwrite_output_dir True \
  --force_image_size 448 \
  --max_dynamic_patch 6 \
  --down_sample_ratio 0.5 \
  --drop_path_rate 0.0 \
  --freeze_llm True \
  --freeze_mlp True \
  --freeze_backbone True \
  --use_llm_lora 16 \
  --vision_select_layer -1 \
  --dataloader_num_workers 4 \
  --bf16 True \
  --num_train_epochs 1 \
  --per_device_train_batch_size ${PER_DEVICE_BATCH_SIZE} \
  --gradient_accumulation_steps ${GRADIENT_ACC} \
  --evaluation_strategy "no" \
  --save_strategy "steps" \
  --save_steps 25804 \
  --save_total_limit 1 \
  --learning_rate 4e-5 \
  --weight_decay 0.01 \
  --warmup_ratio 0.03 \
  --lr_scheduler_type "cosine" \
  --logging_steps 1 \
  --max_seq_length 8192 \
  --do_train True \
  --grad_checkpoint True \
  --group_by_length True \
  --dynamic_image_size True \
  --use_thumbnail True \
  --ps_version 'v2' \
  --deepspeed "zero_stage1_config.json" \
  --report_to "tensorboard" \
  2>&1 | tee -a "/kaggle/working//training_log2.txt"

W0601 10:45:58.156000 3046 torch/distributed/run.py:792] 
W0601 10:45:58.156000 3046 torch/distributed/run.py:792] *****************************************
W0601 10:45:58.156000 3046 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0601 10:45:58.156000 3046 torch/distributed/run.py:792] *****************************************
[2025-06-01 10:46:00,747] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-06-01 10:46:00,747] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-06-01 10:46:00,748] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-06-01 10:46:00,749] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detec

+ GPUS=4
+ BATCH_SIZE=4
+ PER_DEVICE_BATCH_SIZE=1
+ GRADIENT_ACC=1
++ pwd
+ export PYTHONPATH=/kaggle/lib/kagglegym:/kaggle/lib:/kaggle/input/arc-prize-2025:/kaggle/working/InternVL/internvl_chat
+ PYTHONPATH=/kaggle/lib/kagglegym:/kaggle/lib:/kaggle/input/arc-prize-2025:/kaggle/working/InternVL/internvl_chat
+ export MASTER_PORT=34229
+ MASTER_PORT=34229
+ export TF_CPP_MIN_LOG_LEVEL=3
+ TF_CPP_MIN_LOG_LEVEL=3
+ export LAUNCHER=pytorch
+ LAUNCHER=pytorch
+ OUTPUT_DIR=/kaggle/agi/internvl_chat_v2_5/internvl2_5_4b_dynamic_res_2nd_finetune_lora
+ '[' '!' -d /kaggle/agi/internvl_chat_v2_5/internvl2_5_4b_dynamic_res_2nd_finetune_lora ']'
+ torchrun --nnodes=1 --node_rank=0 --master_addr=127.0.0.1 --nproc_per_node=4 --master_port=34229 /kaggle/working/InternVL/internvl_chat/internvl/train/internvl_chat_finetune.py --model_name_or_path OpenGVLab/InternVL3-8B --conv_style internvl2_5 --use_fast_tokenizer False --output_dir /kaggle/agi/internvl_chat_v2_5/internvl2_5_4b_dynamic_res_2nd_finetune

In [7]:
!pip uninstall bitsandbytes -y
!pip install bitsandbytes

Found existing installation: bitsandbytes 0.41.0
Uninstalling bitsandbytes-0.41.0:
  Successfully uninstalled bitsandbytes-0.41.0
  Using cached bitsandbytes-0.46.0-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.46.0-py3-none-manylinux_2_24_x86_64.whl (67.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
internvl-chat 2.0.0 requires bitsandbytes==0.41.0, but you have bitsandbytes 0.46.0 which is incompatible.


In [ ]:

prompt = f'''
You are a shopping assistant who recommends an item of clothing for an image.
the image consists of these clothing elements: {imgs_tokens}
You should suggest another piece of clothing from the semantic category: {sem_cat}.
Make a brief description of the item of clothing that would perfectly match this image.
Your answer should look like this:
'''+'''{'description':'item description'}'''


path2img = '/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/'
for i in tqdm(range(len(val_data))):
    outfit_id = val_data[i]['question'][0].split('_')[0]
    items_list = [j['item_id'] for j in val_df.loc[val_df['set_id']==outfit_id,'items'].tolist().copy()[0]]
    target = val_data[i]['blank_position']

    target_item = items_list[target-1]
    items_list.pop(target-1)

    similarities = []
    answers = []
    for j in val_data[i]['answers']:
        
        outfit_id, index = j.split('_')

        elem = val_df.loc[val_df['set_id']==outfit_id,'items'].tolist().copy()
        elem = elem[0][int(index)-1]['item_id']
        answers.append(elem)

    sem_cat = item_data[str(answers[0])]['semantic_category']
    items_img_paths = [path2img+j+'.jpg' for j in items_list]

    print(sem_cat)
    print(items_img_paths) 
    

In [ ]:
from lmdeploy import pipeline, TurbomindEngineConfig
from lmdeploy.vl import load_image
from lmdeploy.vl.constants import IMAGE_TOKEN
import json
from tqdm import tqdm
import os


path = 'OpenGVLab/InternVL3-8B'
pipe = pipeline(path, backend_config=TurbomindEngineConfig(session_len=8192, tp=4))

path2img = '/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/'
batch_size = 4  

In [1]:
#test
import json
import pandas as pd
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/fill_in_blank_train.json') as f:
    train_data = json.load(f)

with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/train.json') as f:
    train_items = json.load(f)
train_df = pd.DataFrame(train_items)
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/polyvore_item_metadata.json') as f:
    item_data = json.load(f)
    
indexes = []
from tqdm import tqdm

for i in tqdm(range(len(train_data))):
    # cur_test = test_data[i]
    outfit_id = train_data[i]['question'][0].split('_')[0]
    items_list = train_df.loc[train_df['set_id']==outfit_id,'items'].tolist().copy()

    for j in items_list[0]:
        indexes.append(j['item_id'])
    
    # print(outfit_id)
    # test_df 
    # indexes.extend([j.split('_')[0] for j in cur_test['question']])
    # indexes.extend()

100%|██████████| 16995/16995 [00:26<00:00, 629.75it/s]


In [42]:
from tqdm import tqdm
dict_data = {}
for i in tqdm(good_outfits):
    items = i['items']
    for j in items:
        if j['item_id'] not in dict_data.keys():
            dict_data[j['item_id']] = []
        for z in items:
            if (z['item_id']!=j['item_id']):
                dict_data[j['item_id']].append(z['item_id'])

100%|██████████| 12365/12365 [00:00<00:00, 40441.77it/s]


In [44]:
len(dict_data.keys())

43289

In [46]:
import json
with open('structure_data.json','w') as f:
    f.write(json.dumps(dict_data))

In [47]:
pip install lmdeploy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 MB 22.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s et

In [ ]:

import json
import pandas as pd
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/fill_in_blank_valid.json') as f:
    train_data = json.load(f)

with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/valid.json') as f:
    train_items = json.load(f)
train_df = pd.DataFrame(train_items)
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/polyvore_item_metadata.json') as f:
    item_data = json.load(f)
    
indexes = []
from tqdm import tqdm

for i in tqdm(range(len(train_data))):
    # cur_test = test_data[i]
    outfit_id = train_data[i]['question'][0].split('_')[0]
    items_list = train_df.loc[train_df['set_id']==outfit_id,'items'].tolist().copy()

    for j in items_list[0]:
        indexes.append(j['item_id'])
    
    # print(outfit_id)
    # test_df 
    # indexes.extend([j.split('_')[0] for j in cur_test['question']])
    # indexes.extend()

100%|██████████| 3000/3000 [00:01<00:00, 1975.81it/s]


In [ ]:
from lmdeploy import pipeline, TurbomindEngineConfig
from lmdeploy.vl import load_image
from lmdeploy.vl.constants import IMAGE_TOKEN
import json
from tqdm import tqdm
import os


path = 'OpenGVLab/InternVL3-8B'
pipe = pipeline(path, backend_config=TurbomindEngineConfig(session_len=8192, tp=4))


path2img = '/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/'
batch_size = 4  

In [ ]:

batch_size = 4  
def process_images(index_set):
    results = {}
    

    indices = list(index_set)
    
    for i in tqdm(range(0, len(indices), batch_size)):
        batch_indices = indices[i:i+batch_size]
        prompts = []
        images_batch = []
        current_batch = []
        
        for index in batch_indices:
            img_path = os.path.join(path2img, f"{index}.jpg")
            
            try:
                img = load_image(img_path)
                
                prompt = f"{IMAGE_TOKEN}\nDescribe this item of clothing in detail, including its type, style, color, do not add unnecessary words, just a description.  start your text with This is a"
                
                prompts.append(prompt)
                images_batch.append([img])  
                current_batch.append(index)
            except Exception as e:
                print(f"Error loading image {index}: {e}")
                continue
        
        if prompts and images_batch:
            responses = pipe(list(zip(prompts, images_batch)))
            
            for idx, response in zip(current_batch, responses):
                results[idx] = response.text  
    return results

vlm_responses = process_images(list(set(indexes))[:7000])


  0%|          | 0/1750 [00:00<?, ?it/s]

2025-06-01 15:52:25,972 - lmdeploy - WARNING - async_engine.py:645 - GenerationConfig: GenerationConfig(n=1, max_new_tokens=512, do_sample=False, top_p=1.0, top_k=50, min_p=0.0, temperature=0.8, repetition_penalty=1.0, ignore_eos=False, random_seed=None, stop_words=None, bad_words=None, stop_token_ids=[151645], bad_token_ids=None, min_new_tokens=None, skip_special_tokens=True, spaces_between_special_tokens=True, logprobs=None, response_format=None, logits_processors=None, output_logits=None, output_last_hidden_state=None)
2025-06-01 15:52:25,973 - lmdeploy - WARNING - async_engine.py:646 - Since v0.6.0, lmdeploy add `do_sample` in GenerationConfig. It defaults to False, meaning greedy decoding. Please set `do_sample=True` if sampling  decoding is needed


100%|██████████| 1750/1750 [43:24<00:00,  1.49s/it]


In [51]:
len(list(set(indexes)))

14657

In [53]:
with open('data_sorted_valid1.json', 'w', encoding='utf-8') as f:
    json.dump(vlm_responses, f, ensure_ascii=False, indent=4, sort_keys=True)

In [54]:
!export PYTHONPATH=/kaggle/working/InternVL:$PYTHONPATH
!python /kaggle/working/InternVL/internvl_chat/tools/merge_lora.py /kaggle/agi/internvl_chat_v2_5/internvl2_5_4b_dynamic_res_2nd_finetune_lora /kaggle/working/internvl2_5_4b_dynamic_res_2nd_finetune_lora_merged

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Traceback (most recent call last):
  File "/kaggle/working/InternVL/internvl_chat/tools/merge_lora.py", line 4, in <module>
    from internvl.model.internvl_chat import InternVLChatModel
ModuleNotFoundError: No module named 'internvl'


## test

In [2]:
import pandas as pd
import json
desc = []
for i in range(1,3):
    path = f'/kaggle/input/description-data/data_sorted_valid{i}.json'
    with open(path, 'r', encoding='utf-8') as f:
        desc.append( pd.DataFrame(json.load(f),index = ['description']).T)
desc_df = pd.concat(desc)
desc_df

,description
100063042,This is a black leather pencil skirt. It featu...
100263578,This is a denim skirt. It is a mid-length styl...
100380863,This is a golden headpiece with an arch shape....
100665540,"This is a white, short-sleeved t-shirt with a ..."
100679541,This is a black leather jacket with a structur...
...,...
99429004,This is a pair of women's pointed-toe flats. T...
99469217,This is a brown leather tote bag. It features ...
99493678,This is a pair of black sunglasses with a clas...
99526510,This is a black long-sleeve sweatshirt with wh...


In [3]:
desc_df['start_word'] = desc_df.description.apply(lambda x: ' '.join(x.split(' ')[:3]))

In [4]:
desc_df['start_word'].value_counts()[:50]

start_word
This is a     14633
This is an       24
Name: count, dtype: int64

In [5]:
to_del = ['This is a','This is an']

In [6]:
desc_df['description1'] = desc_df.description.apply(lambda x: ' '.join(x.split(' ')[3:] if (' '.join(x.split(' ')[:3]) in to_del) else ''))
desc_df.loc[desc_df['description1'] == '','description1'] = desc_df.loc[desc_df['description1'] == '','description']

In [7]:
desc_df

,description,start_word,description1
100063042,This is a black leather pencil skirt. It featu...,This is a,black leather pencil skirt. It features a smoo...
100263578,This is a denim skirt. It is a mid-length styl...,This is a,denim skirt. It is a mid-length style with a s...
100380863,This is a golden headpiece with an arch shape....,This is a,golden headpiece with an arch shape. It featur...
100665540,"This is a white, short-sleeved t-shirt with a ...",This is a,"white, short-sleeved t-shirt with a round neck..."
100679541,This is a black leather jacket with a structur...,This is a,black leather jacket with a structured design....
...,...,...,...
99429004,This is a pair of women's pointed-toe flats. T...,This is a,pair of women's pointed-toe flats. The shoes a...
99469217,This is a brown leather tote bag. It features ...,This is a,brown leather tote bag. It features a spacious...
99493678,This is a pair of black sunglasses with a clas...,This is a,pair of black sunglasses with a classic aviato...
99526510,This is a black long-sleeve sweatshirt with wh...,This is a,black long-sleeve sweatshirt with white text a...


In [9]:
import json
with open('/kaggle/input/description-data/strucure_val.json', 'r') as f:
    data_rel = json.load(f)

In [11]:
desc_df = desc_df.reset_index()

In [12]:
desc_df = desc_df.drop_duplicates(subset=['index'])

In [13]:
index_list = desc_df['index'].tolist()

In [ ]:
#test
import json
import pandas as pd
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/fill_in_blank_valid.json') as f:
    train_data = json.load(f)

with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/disjoint/valid.json') as f:
    train_items = json.load(f)
train_df = pd.DataFrame(train_items)
with open('/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/polyvore_item_metadata.json') as f:
    item_data = json.load(f)
    
    
outfits = []
from tqdm import tqdm

for i in tqdm(range(len(train_data))):
    outfit_id = train_data[i]['question'][0].split('_')[0]
    items_list = train_df.loc[train_df['set_id']==outfit_id,'items'].tolist().copy()
    desc_items = []
    for j in items_list[0]:
        if (j['item_id'] in index_list):
            desc_items.append(j)
    outfits.append({'id':outfit_id,'items':desc_items})

100%|██████████| 3000/3000 [00:03<00:00, 953.22it/s]


In [ ]:
good_outfits = []
for i in outfits:
    if (len(i['items']) > 0):
        good_outfits.append(i)

2992

In [16]:
from tqdm import tqdm
dict_data = {}
for i in tqdm(good_outfits):
    items = i['items']
    for j in items:
        if j['item_id'] not in dict_data.keys():
            dict_data[j['item_id']] = []
        for z in items:
            if (z['item_id']!=j['item_id']):
                dict_data[j['item_id']].append(z['item_id'])

100%|██████████| 2992/2992 [00:00<00:00, 156585.12it/s]


In [17]:
for i in dict_data.keys():
    dict_data[i] = list(set(dict_data[i]))

In [18]:
len(dict_data.keys())

10982

In [27]:
pip install lmdeploy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 MB 22.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.9 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.0 MB/s et

In [61]:
with open ('/kaggle/working/strucure_val.json','w') as f:
    f.write(json.dumps(dict_data))

In [ ]:
from lmdeploy import pipeline, TurbomindEngineConfig
from lmdeploy.vl import load_image
from lmdeploy.vl.constants import IMAGE_TOKEN
import json
from tqdm import tqdm
import os


path = '/kaggle/working/internvl2_5_4b_dynamic_res_2nd_finetune_lora_merged'
pipe = pipeline(path, backend_config=TurbomindEngineConfig(session_len=8192, tp=4))


path2img = '/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/'


2025-06-02 04:13:54.915166: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748837634.937272     470 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748837634.944001     470 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


FlashAttention2 is not installed.


  0%|          | 0/431 [00:00<?, ?w/s]

  0%|          | 0/123 [00:00<?, ?w/s]

  0%|          | 0/124 [00:00<?, ?w/s]

  0%|          | 0/7 [00:00<?, ?w/s]

2025-06-02 04:14:03,622 - lmdeploy - WARNING - turbomind.py:252 - get 849 model params


[TM][WARNING] [LlamaTritonModel] `max_context_token_num` is not set, default to 8192.
[TM][WARNING] pad vocab size from 151674 to 151676
[TM][WARNING] pad embed size from 151676 to 151676
[TM][WARNING] pad vocab size from 151674 to 151676
[TM][WARNING] pad embed size from 151676 to 151676
[TM][WARNING] pad vocab size from 151674 to 151676
[TM][WARNING] pad embed size from 151676 to 151676
[TM][WARNING] pad vocab size from 151674 to 151676
[TM][WARNING] pad embed size from 151676 to 151676


2025-06-02 04:14:11,353 - lmdeploy - WARNING - tokenizer.py:499 - The token <|action_end|>, its length of indexes [27, 91, 1311, 6213, 91, 29] is over than 1. Currently, it can not be used as stop words


In [ ]:
# разметка теста по батчам
batch_size = 4  
def process_images(index_set):
    results = {}
    
    indices = list(index_set)
    
    for i in tqdm(range(0, len(indices), batch_size)):
        batch_indices = indices[i:i+batch_size]
        prompts = []
        images_batch = []
        current_batch = []
        
        for index in batch_indices:
            img_path = os.path.join(path2img, f"{index}.jpg")
            
            try:
                img = load_image(img_path)

                desc = desc_df.loc[desc_df['index'] == index,'description1'].tolist()[0]
                prompt = f"{IMAGE_TOKEN}\nyou are a shopping assistant. recommend 4 items of clothing for {desc} , you must analyze the description of the clothing, its image. in the answer you must make a detailed description of things that match this item of clothing"
                
                prompts.append(prompt)
                images_batch.append([img])  
                current_batch.append(index)
            except Exception as e:
                print(f"Error loading image {index}: {e}")
                continue
        
        if prompts and images_batch:
            responses = pipe(list(zip(prompts, images_batch)))
            
            for idx, response in zip(current_batch, responses):
                results[idx] = response.text 
    
    return results


vlm_responses = process_images(list(set(dict_data.keys()))[5000:])


100%|██████████| 1496/1496 [1:43:04<00:00,  4.13s/it]


In [22]:
with open('test_rec2.json', 'w', encoding='utf-8') as f:
    json.dump(vlm_responses, f, ensure_ascii=False, indent=4, sort_keys=True)

In [47]:
!kaggle datasets init -p /kaggle/working/internvl2_5_4b_dynamic_res_2nd_finetune_lora_merged

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Data package template written to: /kaggle/working/internvl2_5_4b_dynamic_res_2nd_finetune_lora_merged/dataset-metadata.json


In [154]:
lines = []
with open('/kaggle/working/test_rec1.json') as f:
    lines.extend(f.readlines())
with open('/kaggle/working/test_rec2.json') as f:
    lines.extend(f.readlines())

In [316]:
json_rec = {}
for idx,i in enumerate(lines):
    try:
        index = int(i.split('"goes with this item of clothing: ')[0].replace(':','').replace('"',''))
        text = i.split('"goes with this item of clothing: ')[1][:-3]
        json_rec[index] = text
    except:
        print(idx,i)

0 {

5001 }
5002 {

7011     "179245514": "gothic style, featuring a black and white color scheme. The design includes a black top with a white collar and cuffs, and a white skirt with black trim. The top has a lace-up front and long sleeves. The skirt has a layered, ruffled hem.",

10985 }


In [177]:
json_rec[211862378]

"['pink crossbody bag with a flap closure. It features a gold-tone chain strap and a small, rectangular gold-tone logo on the front. The bag has a smooth texture and a simple, elegant design.', 'pair of women's sandals. They feature a flat sole with a light pink color. The straps are made of a glittery, metallic fabric in shades of pink and purple. The straps are adorned with small, shiny embellishments. The design includes a single strap over the toes and a wider strap around the ankle, with a small buckle for adjustment.', 'pair of sunglasses with a cat-eye frame style. The frame is a light pink color, and the lenses are tinted in a gradient shade of pink. The design features a classic, retro look with a slightly curved top edge.', 'pair of light pink denim shorts. They feature a high waist with a button closure and belt loops. The shorts have two front pockets with buttoned flaps and two back pockets. The hem is slightly frayed, giving a casual, distressed look.']"

In [317]:
json_rec[179245514] = '''['pink crossbody bag with a flap closure. It features a gold-tone chain strap and a small, rectangular gold-tone logo on the front. The bag has a smooth texture and a simple, elegant design.', 'pair of women's sandals. They feature a flat sole with a light pink color. The straps are made of a glittery, metallic fabric in shades of pink and purple. The straps are adorned with small, shiny embellishments. The design includes a single strap over the toes and a wider strap around the ankle, with a small buckle for adjustment.', 'pair of sunglasses with a cat-eye frame style. The frame is a light pink color, and the lenses are tinted in a gradient shade of pink. The design features a classic, retro look with a slightly curved top edge.', 'pair of light pink denim shorts. They feature a high waist with a button closure and belt loops. The shorts have two front pockets with buttoned flaps and two back pockets. The hem is slightly frayed, giving a casual, distressed look.']'''

In [ ]:
с=0
json_rec_list = {}
for i in json_rec.keys():
    try:
        json_rec_list[i] = eval(json_rec[i])
    except:
        json_string = json_rec[i]
        cleaned_string = json_string.strip("'").strip('[]')
        items = re.split(",\s*(?=(?:[^']*'[^']*')*[^']*$)", cleaned_string)
        result = [item.strip().strip("'").replace("\\'", "'") for item in items if item.strip()]

        json_rec_list[i] = result


In [20]:
pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 11.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-

In [80]:
def get_combined_embeddings(model, image,text,  device):
    image_features = model.encode_image(image.to(device))
    text_features = model.encode_text(text.to(device))

    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    combined_features = torch.cat([image_features, text_features],dim=1)
    return image_features, text_features

In [22]:
import open_clip
import torch
from PIL import Image

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:Marqo/marqo-fashionSigLIP')
tokenizer = open_clip.get_tokenizer('hf-hub:Marqo/marqo-fashionSigLIP')


open_clip_model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [266]:
model.cuda()
pass

In [55]:
with open('/kaggle/input/description-data/test_rec1.json') as f:
    json_rec1 = json.load(f)
with open('/kaggle/input/description-data/test_rec2.json') as f:
    json_rec2 = json.load(f)
json_rec = json_rec1.copy()
json_rec.update(json_rec2)

In [56]:
len(desc_filt.index.tolist()),len(json_rec.keys())

(10982, 10982)

In [ ]:
import pandas as pd
import json
desc = []
for i in range(1,3):
    path = f'/kaggle/input/description-data/data_sorted_valid{i}.json'
    with open(path, 'r', encoding='utf-8') as f:
        desc.append( pd.DataFrame(json.load(f),index = ['description']).T)
desc_df = pd.concat(desc)
desc_filt = desc_df.reset_index()
desc_filt = desc_filt.drop_duplicates(subset=['index'],keep='last')
desc_json = desc_filt.set_index('index').to_json()

In [215]:
desc_filt.shape

(10982, 2)

In [ ]:
from torch.utils.data import Dataset
import random

class Dataset(Dataset):
    def __init__(self, data, indexes, img_folder, descriptions, json_rec,transform=None, tokenizer=None, is_train=True):
        self.data = data
        self.indexes = indexes
        self.transform = transform
        self.img_folder = img_folder
        self.descriptions = descriptions
        self.tokenizer = tokenizer
        self.is_train = is_train
        self.json_rec = json_rec

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, idx):
        anchor_id = self.indexes[idx]
        anchor_img = Image.open(f'{self.img_folder}/{anchor_id}.jpg').convert("RGB")
        
        anchor_img = self.transform(anchor_img)
        return anchor_id, anchor_img 

In [302]:
val_ids = desc_filt['index'].tolist()

In [303]:
test_dataset = Dataset(
    dict_data,
    val_ids,
    '/kaggle/input/polyvore-outfit-dataset/polyvore_outfits/images/',
    json.loads(desc_json),
    json_rec = json_rec_list,
    transform=preprocess_val,
    tokenizer=tokenizer,
    is_train=False
)

In [304]:
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim
batch_size=16
device = 'cuda'
val_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_image_embs = []
val_ids = []

In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import torch

val_image_embs = []
val_ids = []
with torch.no_grad():
    for batch in tqdm(val_dataloader):
        anchor_id,anchor_img = batch  
        anchor_img = anchor_img.to(device)
        image_features = model.encode_image(anchor_img)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True) 
        val_image_embs.append(image_features.cpu().numpy())
        val_ids.extend(anchor_id if isinstance(anchor_id, list) else [anchor_id])
val_image_embs = np.vstack(val_image_embs)  

100%|██████████| 687/687 [01:30<00:00,  7.56it/s]


In [309]:
val_ids = desc_filt['index'].tolist()

In [341]:
desc_js = json.loads(desc_json)['description']

In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import torch
from collections import defaultdict


hits_1 = []
hits_5 = []
hits_25 = []
hits_100 = []

all_recs = []
for sample_id_str in json_rec_list.keys():
    sample_id = int(sample_id_str)
    rec_desc_list = json_rec_list[sample_id].copy()
    for rec_text in rec_desc_list:
        all_recs.append((sample_id, rec_text))


batch_size = 128  
hits_1_dict = defaultdict(list)
hits_5_dict = defaultdict(list)
hits_25_dict = defaultdict(list)
hits_100_dict = defaultdict(list)

for i in tqdm(range(0, len(all_recs), batch_size), desc="Обработка батчей"):
    batch = all_recs[i:i + batch_size]
    batch_texts = [rec_text for _, rec_text in batch]  
    batch_sample_ids = [sample_id for sample_id, _ in batch]  

    with torch.no_grad():
        rec_tokenized = tokenizer(batch_texts).to(device)
        text_features = model.encode_text(rec_tokenized)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        text_embs = text_features.cpu().numpy()

    similarities = cosine_similarity(text_embs, val_image_embs)
    for j, sample_id in enumerate(batch_sample_ids):
        sim = similarities[j]
        sorted_indices = np.argsort(sim)[::-1]
        sorted_val_ids = [val_ids[idx] for idx in sorted_indices]

        relevant_ids = dict_data[str(sample_id)] 

        if relevant_ids:
            top_1_ids = sorted_val_ids[:1]
            top_5_ids = sorted_val_ids[:5]
            top_25_ids = sorted_val_ids[:25]
            top_100_ids = sorted_val_ids[:100]

            hit_1 = 1 if any(rel_id in top_1_ids for rel_id in relevant_ids) else 0
            hit_5 = 1 if any(rel_id in top_5_ids for rel_id in relevant_ids) else 0
            hit_25 = 1 if any(rel_id in top_25_ids for rel_id in relevant_ids) else 0
            hit_100 = 1 if any(rel_id in top_100_ids for rel_id in relevant_ids) else 0

            hits_1_dict[sample_id].append(hit_1)
            hits_5_dict[sample_id].append(hit_5)
            hits_25_dict[sample_id].append(hit_25)
            hits_100_dict[sample_id].append(hit_100)

for sample_id in hits_1_dict:
    mean_hit_1 = np.mean(hits_1_dict[sample_id])
    mean_hit_5 = np.mean(hits_5_dict[sample_id])
    mean_hit_25 = np.mean(hits_25_dict[sample_id])
    mean_hit_100 = np.mean(hits_100_dict[sample_id])

    hits_1.append(mean_hit_1)
    hits_5.append(mean_hit_5)
    hits_25.append(mean_hit_25)
    hits_100.append(mean_hit_100)


mean_hit_1 = np.mean(hits_1) if hits_1 else 0
mean_hit_5 = np.mean(hits_5) if hits_5 else 0
mean_hit_25 = np.mean(hits_25) if hits_25 else 0
mean_hit_100 = np.mean(hits_100) if hits_100 else 0

print(f"Итоговая метрика hit@1: {mean_hit_1:.4f}")
print(f"Итоговая метрика hit@5: {mean_hit_5:.4f}")
print(f"Итоговая метрика hit@25: {mean_hit_25:.4f}")
print(f"Итоговая метрика hit@100: {mean_hit_100:.4f}")

Обработка батчей: 100%|██████████| 344/344 [02:31<00:00,  2.27it/s]


Итоговая метрика hit@1: 0.0011
Итоговая метрика hit@5: 0.0055
Итоговая метрика hit@25: 0.0222
Итоговая метрика hit@100: 0.0771


In [ ]:
Итоговая метрика hit@1: 0.0011
Итоговая метрика hit@5: 0.0055
Итоговая метрика hit@25: 0.0222
Итоговая метрика hit@100: 0.0771
finetuned internvl3-8B + marqo-fashionSigLIP для сравнения эмбеддингов

In [346]:
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F

model_path = 'Alibaba-NLP/gte-large-en-v1.5'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).to(device)

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py:   0%|          | 0.00/59.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [ ]:

val_texts = [desc_js[str(sample_id)] for sample_id in val_ids]

val_text_embs = []
batch_size = 16  
with torch.no_grad():
    for i in tqdm(range(0, len(val_texts), batch_size), desc="Computing sample text embeddings"):
        batch_texts = val_texts[i:i + batch_size]
        tokenized = tokenizer(batch_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt').to(device)
        outputs = model(**tokenized)
        text_features = outputs.last_hidden_state[:, 0]  
        text_features = F.normalize(text_features, p=2, dim=1)  
        val_text_embs.append(text_features.cpu().numpy())
val_text_embs = np.vstack(val_text_embs)  

Computing sample text embeddings: 100%|██████████| 687/687 [01:40<00:00,  6.83it/s]


In [ ]:
from collections import defaultdict


all_recs = []
for sample_id_str in json_rec_list.keys():
    sample_id = int(sample_id_str) 
    rec_desc_list = json_rec_list[sample_id_str] 
    for rec_text in rec_desc_list:
        all_recs.append((sample_id, rec_text))


hits_1_dict = defaultdict(list)
hits_5_dict = defaultdict(list)
hits_25_dict = defaultdict(list)
hits_100_dict = defaultdict(list)

batch_size = 128  
for i in tqdm(range(0, len(all_recs), batch_size), desc="Processing recommendation batches"):
    batch = all_recs[i:i + batch_size]
    batch_texts = [rec_text for _, rec_text in batch]
    batch_sample_ids = [sample_id for sample_id, _ in batch]

    with torch.no_grad():
        tokenized = tokenizer(batch_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt').to(device)
        outputs = model(**tokenized)
        text_features = outputs.last_hidden_state[:, 0]
        text_features = F.normalize(text_features, p=2, dim=1)
        text_embs = text_features.cpu().numpy()  

    similarities = cosine_similarity(text_embs, val_text_embs) 

    for j, sample_id in enumerate(batch_sample_ids):
        sim = similarities[j]  
        sorted_indices = np.argsort(sim)[::-1]  
        sorted_val_ids = [val_ids[idx] for idx in sorted_indices]  

        relevant_ids =  dict_data[str(sample_id)]  

        if relevant_ids:
            top_1_ids = sorted_val_ids[:1]
            top_5_ids = sorted_val_ids[:5]
            top_25_ids = sorted_val_ids[:25]
            top_100_ids = sorted_val_ids[:100]

            hit_1 = 1 if any(rel_id in top_1_ids for rel_id in relevant_ids) else 0
            hit_5 = 1 if any(rel_id in top_5_ids for rel_id in relevant_ids) else 0
            hit_25 = 1 if any(rel_id in top_25_ids for rel_id in relevant_ids) else 0
            hit_100 = 1 if any(rel_id in top_100_ids for rel_id in relevant_ids) else 0

            hits_1_dict[sample_id].append(hit_1)
            hits_5_dict[sample_id].append(hit_5)
            hits_25_dict[sample_id].append(hit_25)
            hits_100_dict[sample_id].append(hit_100)

Processing recommendation batches: 100%|██████████| 344/344 [09:46<00:00,  1.71s/it]


In [ ]:
hits_1 = []
hits_5 = []
hits_25 = []
hits_100 = []

for sample_id in hits_1_dict:
    mean_hit_1 = np.mean(hits_1_dict[sample_id])
    mean_hit_5 = np.mean(hits_5_dict[sample_id])
    mean_hit_25 = np.mean(hits_25_dict[sample_id])
    mean_hit_100 = np.mean(hits_100_dict[sample_id])

    hits_1.append(mean_hit_1)
    hits_5.append(mean_hit_5)
    hits_25.append(mean_hit_25)
    hits_100.append(mean_hit_100)

mean_hit_1 = np.mean(hits_1) if hits_1 else 0
mean_hit_5 = np.mean(hits_5) if hits_5 else 0
mean_hit_25 = np.mean(hits_25) if hits_25 else 0
mean_hit_100 = np.mean(hits_100) if hits_100 else 0

print(f"Итоговая метрика hit@1: {mean_hit_1:.4f}")
print(f"Итоговая метрика hit@5: {mean_hit_5:.4f}")
print(f"Итоговая метрика hit@25: {mean_hit_25:.4f}")
print(f"Итоговая метрика hit@100: {mean_hit_100:.4f}")

Итоговая метрика hit@1: 0.0012
Итоговая метрика hit@5: 0.0057
Итоговая метрика hit@25: 0.0214
Итоговая метрика hit@100: 0.0682


In [ ]:
Итоговая метрика hit@1: 0.0012
Итоговая метрика hit@5: 0.0057
Итоговая метрика hit@25: 0.0214
Итоговая метрика hit@100: 0.0682
finetuned internvl3-8B + gte-large-en-v1.5 для сравнения эмбеддингов